In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch.utils.data import DataLoader
from align_anything.datasets.text_to_text.preference import PreferenceBatch, PreferenceDataset
from torch.utils.data.distributed import DistributedSampler
from align_anything.configs.template import ChatTemplate
from align_anything.models.pretrained_model import load_pretrained_models
import json
from tqdm import tqdm
device = "cuda" # the device to load the model onto


prompt = "Give me a short introduction to large language model."





/home/admin01/anaconda3/envs/align-anything/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-06-02 02:18:08,332] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/admin01/anaconda3/envs/align-anything/bin/../lib/gcc/x86_64-conda-linux-gnu/11.2.0/../../../../x86_64-conda-linux-gnu/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/admin01/anaconda3/envs/align-anything/bin/../lib/gcc/x86_64-conda-linux-gnu/11.2.0/../../../../x86_64-conda-linux-gnu/bin/ld: /home/admin01/anaconda3/envs/align-anything/lib/libcufile.so: undefined reference to `dlvsym'
/home/admin01/anaconda3/envs/align-anything/bin/../lib/gcc/x86_64-conda-linux-gnu/11.2.0/../../../../x86_64-conda-linux-gnu/bin/ld: /home/admin01/anaconda3/envs/align-anything/lib/libcufile.so: undefined reference to `dlopen'
/home/admin01/anaconda3/envs/align-anything/bin/../lib/gcc/x86_64-conda-linux-gnu/11.2.0/../../../../x86_64-conda-linux-gnu/bin/ld: /home/admin01/anaconda3/envs/align-anything/lib/libcufile.so: undefined reference to `dlclose'
/home/admin01/anaconda3/envs/align-anything/bin/../lib/gcc/x86_64-conda-linux-gnu/11.2.0/../../../../

In [8]:
def get_response_from_model(model,tokenizer,prompt):
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)
    # generated_ids:Tensor其中包含输入数据，可以直接被用于reward_model打分，这一部分通过下面验证，
    # 可以发现用于训练reward_model的input_ids数据和generated_ids是一个结构的
    # 因为它们decode后都是包含前置system、输入user、回答response三个部分
    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512
    )
    # generated_ids = [
    #     output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    # ]
    # 把回答全部输出出来，更美观
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return generated_ids,response

def get_rewards_from_two_model(model1,tokenizer1,model2,tokenizer2,prompt,model_reward):
    generated_ids_1,response_1=get_response_from_model(model1,tokenizer1,prompt)
    generated_ids_2,response_2=get_response_from_model(model2,tokenizer2,prompt)
    output_1=model_reward(**{'input_ids':generated_ids_1})
    output_2=model_reward(**{'input_ids':generated_ids_2})
    return response_1,response_2,output_1.end_scores,output_2.end_scores

In [9]:
# 原始模型提取
model_raw = AutoModelForCausalLM.from_pretrained(
    "../../../scripts/qwen2_5/Qwen2.5-0.5B-Instruct",
    torch_dtype="auto",
    device_map="auto"
)
tokenizer_raw = AutoTokenizer.from_pretrained("../../../scripts/qwen2_5/Qwen2.5-0.5B-Instruct")

# dpo强化学习训练后的模型提取
model_dpo = AutoModelForCausalLM.from_pretrained(
    "../../../scripts/outputs/qwen_2_5_dpo/slice_end",
    torch_dtype="auto",
    device_map="auto"
)
tokenizer_dpo = AutoTokenizer.from_pretrained("../../../scripts/outputs/qwen_2_5_dpo/slice_end")

# 加载奖励模型model_reward
model_reward, tokenizer_reward, processor_reward = load_pretrained_models(
    '../../../scripts/outputs/qwen_2_5_rm/slice_end',
    model_max_length=512,
    padding_side='right',
    trust_remote_code=True,
    is_reward_model=True,
    processor_kwargs=None,
    auto_device_mapping=True
)



Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Sliding Window Attention is enabled but not implemented for `eager`; unexpected results may be encountered.
/home/admin01/Shizexi/align-anything/align_anything/models/pretrained_model.py:309: RuntimeWarning: The tokenizer vocabulary size (151665) is different from the model embedding size (151936) before resizing.
  resize_tokenizer_embedding(tokenizer=tokenizer, model=model)
/home/admin01/Shizexi/align-anything/align_anything/models/pretrained_model.py:309: RuntimeWarning: The tokenizer vocabulary size (151665) is different from the model embedding size (151936) after resizing.
  resize_tokenizer_embedding(tokenizer=tokenizer, model=model)


In [10]:
# 这是根据特定的prompt生成的回复
response_raw,response_dpo,reward_raw,reward_dpo=get_rewards_from_two_model(model_raw,tokenizer_raw,model_dpo,tokenizer_dpo,prompt,model_reward)
print(reward_raw[0].item())
print(response_raw)
print(reward_dpo[0].item())
print(response_dpo)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


-2.203125
system
You are a helpful assistant.
user
Give me a short introduction to large language model.
assistant
A large language model (LLM) is a type of artificial intelligence that can generate human-like text based on the input provided by a user or an AI system. These models are designed to understand and respond to complex questions, provide accurate information, and perform various tasks such as translation, summarization, and question answering. They are typically trained using large amounts of data, which enables them to learn from many examples and improve their performance over time. LLMs have become increasingly important in fields such as natural language processing, machine learning, and computer vision, where they are used for a wide range of applications, including automated translations, document generation, and speech synthesis.
-1.765625
system
You are a helpful assistant.
user
Give me a short introduction to large language model.
assistant
A large language model, 

In [11]:
import pandas as pd

df = pd.read_json("../../../assets/text_to_text/hw/val.json", lines=True)  
data_batch=[]
for idx, question in enumerate(df['question']):
    data_batch.append(question)
    if idx>=5:
        break

In [14]:

# 下面应该根据特定的eval数据集批量生成回复并计算reward
def get_batch_rewards_from_two_model(model1,tokenizer1,model2,tokenizer2,data_batch,model_reward,output_file='version1.json'):
    results=[]
    for prompt in tqdm(data_batch, desc="Evaluating prompts"):
        response_raw,response_dpo,reward_raw,reward_dpo=\
            get_rewards_from_two_model(model1,tokenizer1,model2,tokenizer2,prompt,model_reward)
        result = {
            "prompt": prompt,
            "response_raw": response_raw,
            "response_dpo": response_dpo,
            "reward_raw": reward_raw[0].item(),
            "reward_dpo": reward_dpo[0].item()
        }

        results.append(result)

    # 写入 JSON 文件
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(results, f, indent=4, ensure_ascii=False)

    print(f"已保存结果到 {output_file}")
    return results
        

In [15]:
res=get_batch_rewards_from_two_model(model_raw,tokenizer_raw,model_dpo,tokenizer_dpo,data_batch[:3],model_reward,output_file='version1.json')

Evaluating prompts: 100%|██████████| 3/3 [00:53<00:00, 17.95s/it]

已保存结果到 version1.json


In [ ]:
infer_batch = lambda batch: {k: v.to(device) for k, v in batch.items() if k != 'meta_info'}
formatter=tokenizer_dpo
custom_formatter = (
    model_dpo.apply_chat_template
    if hasattr(model_dpo, 'apply_chat_template')
    else None
)
train_template = ChatTemplate(
    formatter, 'HOMEWORK', custom_formatter
)
train_dataset = PreferenceDataset(
    path='../../../assets/text_to_text/hw',
    template=train_template,
    tokenizer=tokenizer_dpo,
    split='train',
)

eval_dataset = PreferenceDataset(
    path='../../../assets/text_to_text/hw',
    template=train_template,
    tokenizer=tokenizer_dpo,
    split='validation',
)

train_dataloader = DataLoader(
    train_dataset,
    collate_fn=train_dataset.get_collator(),
    #sampler=DistributedSampler(train_dataset, shuffle=True),
    batch_size=1,
)
eval_dataloader = DataLoader(
    train_dataset,
    collate_fn=train_dataset.get_collator(),
    #sampler=DistributedSampler(train_dataset, shuffle=True),
    batch_size=1,
)

Generating train split: 30430 examples [00:00, 33753.81 examples/s]
Generating validation split: 1000 examples [00:00, 37582.70 examples/s]
Filtering valid indices: 100%|██████████| 1000/1000 [00:00<00:00, 5698.76it/s]


/home/admin01/Shizexi/align-anything/align_anything/models/pretrained_model.py:309: RuntimeWarning: The tokenizer vocabulary size (151665) is different from the model embedding size (151936) before resizing.
  resize_tokenizer_embedding(tokenizer=tokenizer, model=model)
/home/admin01/Shizexi/align-anything/align_anything/models/pretrained_model.py:309: RuntimeWarning: The tokenizer vocabulary size (151665) is different from the model embedding size (151936) after resizing.
  resize_tokenizer_embedding(tokenizer=tokenizer, model=model)


In [13]:
print(model_reward(**{'input_ids':test_ids}).end_scores)

tensor([[1.1172]], device='cuda:0', grad_fn=<ToCopyBackward0>)


In [ ]:
rewards=[]
num_correct_predictions=0
num_total_predictions=0
for batch in eval_dataloader:
    output = model_reward(**infer_batch(batch))
    
    response = tokenizer_reward.batch_decode(batch['input_ids'], skip_special_tokens=True)
    print(response)
    print('batch',batch)
    print('output',output.end_scores)
    for i,j in batch.items():
        print(i)
    break
    # end_scores = output.end_scores
    # higher_end_rewards, lower_end_rewards = end_scores.squeeze(dim=-1).chunk(
    #     chunks=2, dim=0
    # )
    # batch_size = higher_end_rewards.size(0)
    # num_correct_predictions += (higher_end_rewards > lower_end_rewards).sum()
    # num_total_predictions += batch_size

    # rewards.extend([higher_end_rewards, lower_end_rewards])

['system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.\nuser\nCan a Muslim person use pork flavoring?\nassistant\nNo, a Muslim person cannot use pork flavoring as it is haram (prohibited) for them according to their religion. Pork is forbidden in the Quran and any ingredient containing pork flavoring is also off limits. It is important for Muslims to abstain from all types of haram food and drink, such as alcohol and other non-halal meats. Pork flavoring should not be used by Muslims since it will lead them away from their faith.\n', 'system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.\nuser\nCan a Muslim person use pork flavoring?\nassistant\nNo, a Muslim person cannot use pork flavoring as it is forbidden in Islam. Muslims consider pork to be impure and forbidden, and thus, consuming it or using it in any way is not allowed. Furthermore, touching or consuming any pork products is also forbidden for Muslims.\n']
batch {'meta_info': {'re

In [9]:
print(len(response))

2


In [5]:
for idx ,batch, in enumerate(train_dataloader):
    print(batch)

{'meta_info': {'response_lens': [258, 258, 156, 43, 94, 94, 57, 68, 51, 41, 82, 212, 32, 46, 125, 88, 19, 275, 75, 14, 237, 45, 41, 18, 27, 73, 10, 144, 27, 21, 50, 71]}, 'input_ids': tensor([[151644,   8948,    198,  ...,     13, 151645,    198],
        [151643, 151643, 151643,  ...,     13, 151645,    198],
        [151643, 151643, 151643,  ...,      0, 151645,    198],
        ...,
        [151643, 151643, 151643,  ...,     13, 151645,    198],
        [151643, 151643, 151643,  ...,     13, 151645,    198],
        [151643, 151643, 151643,  ...,     13, 151645,    198]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        ...,
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1]], device='cuda:0')}
{'meta_info': {'response_lens': [59, 349, 63, 49, 70, 55, 54, 70, 43, 41, 91, 82, 219, 134, 109, 99, 32, 167, 68, 33, 79, 16, 105, 97, 49, 

In [14]:

data = []

with open('../../../assets/text_to_text/hw/val.json', 'r', encoding='utf-8') as f:
    for line in f:
        line = line.strip()
        if line:  # 跳过空行
            obj = json.loads(line)
            data.append(obj['question'])


In [16]:
res=get_batch_rewards_from_two_model(model_raw,tokenizer_raw,model_dpo,tokenizer_dpo,data,model_reward,'mam.json')

Evaluating prompts:   0%|          | 2/1000 [00:46<6:25:29, 23.18s/it]


KeyboardInterrupt: 